In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime


In [57]:
class MNIST(object):
    def __init__(self,batch_size,split=False,size=None,shuffle=False):
        train,test=tf.keras.datasets.mnist.load_data()
        X,y=train
        self.N=X.shape[0]
       
        X=X.astype(np.float32)/255
        X=X.reshape((*X.shape,1))
        X=np.concatenate((X,X,X),axis=3)
        
        if split is True and size is not  None:
            self.X_train=X[:self.N-size,:,:,:]
            self.y_train=y[:self.N-size]
            self.X_val=X[self.N-size:,:,:,:]
            self.y_val=y[self.N-size:]
        self.X_train,self.y_train=X,y
        
        self.batch_size,self.shuffle=batch_size,shuffle
        
    def __iter__(self):
        N,B=self.X_train.shape[0],self.batch_size
        idxs=np.array(N)
        if self.shuffle:
            np.random.shuffle(indxs)
        return iter((self.X_train[i:i+B],self.y_train[i:i+B]) for i in range(0,N,B))
    def get_val(self):
        if 'X_val' in dir(self):
            return self.X_val,self.y_val
        else:
            return None
    def get_train(self):
        return self.X_train,self.y_train

In [58]:
class CNet(tf.keras.Model):
    def __init__(self,input_shape,num_classes):
        super(CNet,self).__init__()
        self.num_classes=num_classes
        self.net=tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'),
            tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=num_classes)])
        
    def call(self,x):
        scores=self.net(x)
        return scores

In [59]:
log_dir="CNet:{}".format(datetime.now())

In [63]:
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=5,
                                          write_graph=False,write_images=True,
                                          embeddings_freq=0)

In [64]:
def train(model,datasets,learning_rate=1e-1,verbose=True):
    optimizer=tf.keras.optimizers.(learning_rate=learning_rate)
    accuracy=tf.keras.metrics.SparseCategoricalAccuracy()
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    net=model
    net.compile(optimizer=optimizer, loss=loss,metrics=[accuracy])
    X_train,y_train=datasets.get_train()
    X_val,y_val=datasets.get_val()
    
    net.fit(x=X_train,y=y_train,batch_size=64,epochs=10,
           callbacks=[tensorboard],validation_data=(X_val,y_val),validation_freq=2)
    
    

In [65]:
datasets=MNIST(1024,split=True,size=2000)
model=CNet(input_shape=(28,28,3),num_classes=10)
train(model,datasets,learning_rate=1e-1)

Train on 60000 samples, validate on 2000 samples
Epoch 1/10
60000/60000 [==============================] - 27s 455us/sample - loss: 0.9377 - sparse_categorical_accuracy: 0.7246
Epoch 2/10
60000/60000 [==============================] - 27s 458us/sample - loss: 0.4638 - sparse_categorical_accuracy: 0.8644 - val_loss: 0.2612 - val_sparse_categorical_accuracy: 0.9270
Epoch 3/10
60000/60000 [==============================] - 30s 497us/sample - loss: 0.4211 - sparse_categorical_accuracy: 0.8806
Epoch 4/10
60000/60000 [==============================] - 32s 530us/sample - loss: 0.4761 - sparse_categorical_accuracy: 0.8768 - val_loss: 0.2791 - val_sparse_categorical_accuracy: 0.9320
Epoch 5/10
60000/60000 [==============================] - 29s 488us/sample - loss: 0.3898 - sparse_categorical_accuracy: 0.8937
Epoch 6/10
60000/60000 [==============================] - 30s 508us/sample - loss: 0.3930 - sparse_categorical_accuracy: 0.8963 - val_loss: 0.2637 - val_sparse_categorical_accuracy: 0.9335


In [68]:
log_dir

'CNet:2019-09-26 00:42:55.114197'